# <ins> <center> Futuristic Airbnb ETL </center> </ins>
<hr style="boder:2px">

 <b>
Airbnb is a home-sharing platform, which provides a platform for hosts to accommodate guests with short-term lodging and tourism-related activities. Guest3s can search for lodging using filters such as lodging type, dates, location, and price Guests have the ability to search for specific types of homes, such as bed and breakfasts, unique homes, and vacation homes before booking, users must provide personal and payment information. Some hosts also require a scan of government-issued identification before accepting a reservation Guests have the ability to chat with hosts through a secure messaging system Hosts provide prices and other details for their rental or event listings, such as the allowed number of guests, home type, rules, and amenities. The host, with recommendations from Airbnb, determines pricing. Hosts and guests have the ability to leave reviews about the experience. 
</b>

<hr>

<hr>

In [2]:
#Importing Dependencies

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import json
import requests
import gmaps
import gmaps
import gmaps.datasets
from math import sin, cos, sqrt, atan2, radians

In [4]:
#reading popular place in all the islands 
file_one_df = pd.read_csv("../Dataset-ETL/Hawaii/place.csv", encoding="utf8")
file_one_df.head(10)


,place,information,lat,lng
0,Hawaii Volcanoes National Park,This is one of the most geologically interesti...,19.419400,-155.288500
1,Thurston Lava Tube,NaN,19.413800,-155.238600
2,Jaggar Museum,NaN,19.419900,-155.288000
3,Punalu'u State Park,NaN,19.135758,-155.504787
4,Waipio Valley and Overlook,This incredibly beautiful valley on the northe...,20.115100,-155.586400
5,Mauna Kea,"Mauna Kea is Hawaii's highest mountain, home t...",19.820600,-155.468100
6,Kona Coffee Living History Farm,This is the nation's only living history museu...,19.491000,-155.914300
7,Pu'uhonua o Honaunau National Historical Park ...,This park is a great place to learn about isla...,19.640000,-155.996900
8,Akaka Falls State Park and Kahuna Falls,Akaka Falls State Park near Hilo is home to bo...,19.853900,-155.152186
9,Akaka Falls,NaN,19.853250,-155.156233


### <ins> Store Listings CSV into DataFrame </ins>

In [5]:
Original_Listings_df = pd.read_csv("../Dataset-ETL/Hawaii/listings.csv", encoding="utf8")
Original_Listings_df.head()


//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (61,62,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3002,https://www.airbnb.com/rooms/3002,20190912153025,2019-09-12,"OCEAN VIEW 1BLK2BEACH,LANAI A/C,FULL KITCH,CEN...",women or couples only. No men alone. Will Dedu...,"**Aloha! I live in a nice, small one bedroom u...",women or couples only. No men alone. Will Dedu...,none,Waikiki has nice weather 90% percent if the ye...,...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,2.09
1,5065,https://www.airbnb.com/rooms/5065,20190912153025,2019-09-12,MAUKA BB,"Perfect for your vacation, Staycation or just ...",Mauka B & B is 400 square feet studio with a p...,"Perfect for your vacation, Staycation or just ...",none,Neighbors here are friendly but are not really...,...,f,f,moderate,f,f,2,2,0,0,0.46
2,5269,https://www.airbnb.com/rooms/5269,20190912153025,2019-09-12,Upcountry Hospitality in the 'Auwai Suite,"The 'Auwai Suite is a lovely, self-contained a...","The ‘Auwai Suite is a lovely, self-contained a...","The 'Auwai Suite is a lovely, self-contained a...",none,"We are located on the ""sunny side"" of Waimea, ...",...,t,f,strict_14_with_grace_period,f,f,7,4,2,0,0.05
3,5387,https://www.airbnb.com/rooms/5387,20190912153025,2019-09-12,Hale Koa Studio & 1 Bedroom Units!!,This Wonderful Spacious Studio apt/flat is in ...,"Hale Koa Duplex: This duplex has a wonderful, ...",This Wonderful Spacious Studio apt/flat is in ...,none,IN a Farm belt area with small commercial farm...,...,t,f,strict_14_with_grace_period,f,f,3,3,0,0,1.36
4,5389,https://www.airbnb.com/rooms/5389,20190912153025,2019-09-12,Keauhou Villa,It is less than 10 minute walk to the Keauhou ...,"Location: Keauhou, Kona Coast, Big Island, Haw...",It is less than 10 minute walk to the Keauhou ...,none,It is less than 10 minute walk to the Keauhou ...,...,f,f,flexible,f,f,3,3,0,0,0.21


### Making a Copy of the Original DataFrame to Listings_df

In [6]:
Listings_df = Original_Listings_df.copy()

In [1]:
# Listings_df.head()

In [7]:
pd.set_option('display.max_columns', len(Listings_df.columns)) 
pd.set_option('display.max_rows', 1000)

In [8]:
#Determining the Null Value Data from the main dataset
round(( Listings_df.isna().sum()/len(Listings_df) ) * 100).sort_values(ascending=False)

xl_picture_url                                  100.0
thumbnail_url                                   100.0
host_acceptance_rate                            100.0
jurisdiction_names                              100.0
medium_url                                      100.0
square_feet                                      99.0
monthly_price                                    95.0
weekly_price                                     94.0
license                                          56.0
transit                                          47.0
access                                           45.0
neighborhood_overview                            43.0
interaction                                      43.0
notes                                            41.0
house_rules                                      29.0
host_about                                       28.0
review_scores_value                              25.0
review_scores_location                           25.0
review_scores_communication 

### Dropping Unneccessary/Similar/Null Columns(Greater than 90%) from the Listings Data frame

In [9]:
Listings_df = Listings_df.drop (['square_feet','weekly_price','monthly_price','scrape_id','last_scraped','space',
                                 'experiences_offered','notes','interaction','thumbnail_url','medium_url','picture_url',
                                 'xl_picture_url','host_acceptance_rate','host_total_listings_count','host_verifications',
                                 'minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights',
                                 'maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated',
                                 'calendar_last_scraped','license','jurisdiction_names','calculated_host_listings_count',
                                 'calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
                                 'calculated_host_listings_count_shared_rooms','neighborhood_overview'], axis =1)


### <ins>Futuristic_Airbnb_distance_df </ins> <hr>

In [11]:
Futuristic_Airbnb_distance_df=Listings_df[['host_id','neighbourhood','latitude','longitude','price']].copy()

In [12]:
Futuristic_Airbnb_distance_df.head()

,host_id,neighbourhood,latitude,longitude,price
0,3364,Oʻahu,21.27603,-157.82210,$69.00
1,7257,Hamakua Coast,20.04095,-155.43251,$85.00
2,7620,Island of Hawaiʻi,20.02621,-155.70145,$97.00
3,7878,Island of Hawaiʻi,19.43119,-155.88079,$92.00
4,7878,Kailua/Kona,19.56413,-155.96347,$299.00


In [13]:
round(( Futuristic_Airbnb_distance_df.isna().sum()/len(Futuristic_Airbnb_distance_df) ) * 100).sort_values(ascending=False)

price            0.0
longitude        0.0
latitude         0.0
neighbourhood    0.0
host_id          0.0
dtype: float64

In [14]:
Futuristic_Airbnb_distance_df["price"] = pd.to_numeric( Futuristic_Airbnb_distance_df["price"].replace({'\$': '', ',': ''}, regex=True))

In [15]:
Futuristic_Airbnb_distance_df.head()

,host_id,neighbourhood,latitude,longitude,price
0,3364,Oʻahu,21.27603,-157.82210,69.0
1,7257,Hamakua Coast,20.04095,-155.43251,85.0
2,7620,Island of Hawaiʻi,20.02621,-155.70145,97.0
3,7878,Island of Hawaiʻi,19.43119,-155.88079,92.0
4,7878,Kailua/Kona,19.56413,-155.96347,299.0


In [18]:
import numpy as np

def distance_np(lon1, lat1, lon2, lat2):
    
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

for index, row in file_one_df.iterrows(): 
    #iterating through island places csv
    place_lat=row["lat"]
    place_lng=row["lng"]
    place=row["place"]
    print(place)
    Futuristic_Airbnb_distance_df[place] =distance_np(Futuristic_Airbnb_distance_df['latitude'],\
                                                         Futuristic_Airbnb_distance_df['longitude'],\
                                                         place_lat,place_lng)
Futuristic_Airbnb_distance_df=Futuristic_Airbnb_distance_df.sort_values('price',ascending=False)
Futuristic_Airbnb_distance_df = Futuristic_Airbnb_distance_df.reset_index(drop=True)
Futuristic_Airbnb_distance_df.head(10)


Hawaii Volcanoes National Park
Thurston Lava Tube
Jaggar Museum
Punalu'u State Park
Waipio Valley and Overlook
Mauna Kea
Kona Coffee Living History Farm
Pu'uhonua o Honaunau National Historical Park (Kona Coast)
Akaka Falls State Park and Kahuna Falls
Akaka Falls
Kahuna Falls
Rainbow Falls
PE’EPE’E FALLS
UMAUMA FALLS
HI’ILAWE WATERFALL (WAIPI’O VALLEY)
Hawaii Tropical Botanical Garden
Hapuna Beach State Park
Waikoloa Beach Resort and Petroglyph Park
Ahu'ena Heiau
Kailua Kona and Magic Sands Beach
Hilo Farmers Market
Kealakekua Bay State Historical Park
Captain James Cook Monument
Hikiau Heiau
Parker Ranch (Kona Coast)
Kohala Mountains
Kamanu Charters
Punalu’u Black Sand Beach
Honl’s Beach
Kahalu’u Beach Park
Makalawena Beach
Waialea Beach
Papakolea
Kauna’oa Beach, aka Mauna Kea Beach
Hapuna Beach State Park
Maniniowali Beach, aka Kua Bay Beach
Polihua Beach
Mike Carroll Gallery
Hulopoe Beach
Lanai Cat Sanctuary
Kaunolu
Sweetheart Rock
Keomuku
Garden of the Gods
Lopa Beach
Halawa Valley

,host_id,neighbourhood,latitude,longitude,price,Hawaii Volcanoes National Park,Thurston Lava Tube,Jaggar Museum,Punalu'u State Park,Waipio Valley and Overlook,Mauna Kea,Kona Coffee Living History Farm,Pu'uhonua o Honaunau National Historical Park (Kona Coast),Akaka Falls State Park and Kahuna Falls,Akaka Falls,Kahuna Falls,Rainbow Falls,PE’EPE’E FALLS,UMAUMA FALLS,HI’ILAWE WATERFALL (WAIPI’O VALLEY),Hawaii Tropical Botanical Garden,Hapuna Beach State Park,Waikoloa Beach Resort and Petroglyph Park,Ahu'ena Heiau,Kailua Kona and Magic Sands Beach,Hilo Farmers Market,Kealakekua Bay State Historical Park,Captain James Cook Monument,Hikiau Heiau,Parker Ranch (Kona Coast),Kohala Mountains,Kamanu Charters,Punalu’u Black Sand Beach,Honl’s Beach,Kahalu’u Beach Park,Makalawena Beach,Waialea Beach,Papakolea,"Kauna’oa Beach, aka Mauna Kea Beach","Maniniowali Beach, aka Kua Bay Beach",Polihua Beach,Mike Carroll Gallery,Hulopoe Beach,Lanai Cat Sanctuary,Kaunolu,Sweetheart Rock,Keomuku,Garden of the Gods,Lopa Beach,Halawa Valley,Papohaku Beach Park,Kalaupapa National Historical Park,Kamakou Preserve,...,WAIANAPANAPA STATE PARK,MOLOKONI CRATER,KAANAPALI BEACH,WAILEA BEACH,KAPALUA BEACH,NAPILI BEACH,HANA HIGHWAY,Ocean Farm (Vodka farm),HAMOA BEACH,LAHAINA JODO MISSION,NAKALELE BLOWHOLE,Waikiki Beach,Duke Kahanamoku Lagoon,The Royal Hawaiian,Iolani Palace,Aliʻiōlani Hale,Diamond Head Crater Hike,Hanauma Bay,Tantalus Lookout,Lyon Arboretum,Moanalua Gardens,Pearl Harbor,Ko Olina Lagoons,Halona Blowhole Lookout,Makapu'u Lookout,Bellows Beach,Lanikai Beach,Ho'omaluhia Botanical Garden,Byodo-In Temple,Macadamia Nut Farm Outlet,Kualoa Ranch,Kualoa Beach Park,Dole Pineapple Plantation,Historic Hale'iwa Town,Turtle Beach,Waimea Bay Beach,Banzai Pipeline,Sunset Beach,Waimea Falls,Manoa Falls,Lulumahu Falls,Jackass Ginger Pool,Likeke Falls,Kahana Bay Beach Park,Malaekahana Beach,Sandy Beach,Pokai Bay,Ala Moana Beach Park,Makapuu Beach Park,Bellows Field Beach Park,Makaha Beach Park,Diamond Head Beach Park,Nanakuli Beach Park
0,51017586,Kihei/Wailea,20.68985,-156.43611,24999.0,181.275790,185.596359,181.278679,188.650675,110.999200,139.112554,134.960714,117.384313,165.935984,165.583882,165.748153,177.224418,175.476712,164.398033,111.650145,173.510801,98.231133,98.404286,117.465172,117.383356,179.262978,135.592491,134.967566,135.742585,107.937098,100.734956,113.399653,188.646325,118.870254,124.386841,102.006314,98.686783,198.196195,97.281829,101.416582,70.916060,55.494632,51.525959,59.519523,58.867104,51.015276,55.959290,65.208398,43.396738,58.813765,103.228458,103.228458,64.175962,...,49.155314,8.908354,37.122808,1.049726,40.654774,38.979170,34.047555,14.294331,49.931830,34.103774,38.321926,165.902881,167.327849,165.388121,170.251523,170.336478,162.956378,151.808705,166.797591,165.399957,175.409211,180.591605,198.444522,150.637015,149.731722,157.089114,159.342745,168.008122,172.692049,178.315473,177.431364,177.328811,197.522056,206.692208,202.284236,205.403579,205.588376,204.826497,204.268418,165.534775,167.280899,167.940009,165.983179,182.860303,194.931543,150.335980,209.323430,168.470707,150.137917,157.089114,214.052747,162.964757,202.320451
1,147713325,Leeward Side,21.34109,-158.11871,24999.0,370.637526,375.633334,370.657484,367.568127,308.020218,332.969293,309.554943,293.076336,362.878895,362.499143,362.710534,373.071825,371.127631,361.825018,308.338023,370.362297,289.857268,288.321787,293.092954,293.073763,375.324216,309.340757,308.496443,309.664661,302.570185,296.117954,288.851723,367.564363,294.570900,299.476704,281.273111,289.980801,368.157886,289.236519,282.202797,127.621959,143.358528,148.979276,140.242516,142.635584,149.631332,142.766876,133.420983,155.272017,154.620381,97.889499,97.889499,140.987115,...,241.893100,194.525183,163.890339,198.060698,165.152736,164.401046,222.262078,201.995170,245.211984,165.921779,173.055809,33.007115,31.621752,33.489889,29.097777,28.971037,35.865947,47.798041,33.006515,35.259845,25.024092,20.368697,1.368647,49.455

In [20]:
 Futuristic_Airbnb_distance_df.head(10)

,host_id,neighbourhood,latitude,longitude,price,Hawaii Volcanoes National Park,Thurston Lava Tube,Jaggar Museum,Punalu'u State Park,Waipio Valley and Overlook,Mauna Kea,Kona Coffee Living History Farm,Pu'uhonua o Honaunau National Historical Park (Kona Coast),Akaka Falls State Park and Kahuna Falls,Akaka Falls,Kahuna Falls,Rainbow Falls,PE’EPE’E FALLS,UMAUMA FALLS,HI’ILAWE WATERFALL (WAIPI’O VALLEY),Hawaii Tropical Botanical Garden,Hapuna Beach State Park,Waikoloa Beach Resort and Petroglyph Park,Ahu'ena Heiau,Kailua Kona and Magic Sands Beach,Hilo Farmers Market,Kealakekua Bay State Historical Park,Captain James Cook Monument,Hikiau Heiau,Parker Ranch (Kona Coast),Kohala Mountains,Kamanu Charters,Punalu’u Black Sand Beach,Honl’s Beach,Kahalu’u Beach Park,Makalawena Beach,Waialea Beach,Papakolea,"Kauna’oa Beach, aka Mauna Kea Beach","Maniniowali Beach, aka Kua Bay Beach",Polihua Beach,Mike Carroll Gallery,Hulopoe Beach,Lanai Cat Sanctuary,Kaunolu,Sweetheart Rock,Keomuku,Garden of the Gods,Lopa Beach,Halawa Valley,Papohaku Beach Park,Kalaupapa National Historical Park,Kamakou Preserve,...,WAIANAPANAPA STATE PARK,MOLOKONI CRATER,KAANAPALI BEACH,WAILEA BEACH,KAPALUA BEACH,NAPILI BEACH,HANA HIGHWAY,Ocean Farm (Vodka farm),HAMOA BEACH,LAHAINA JODO MISSION,NAKALELE BLOWHOLE,Waikiki Beach,Duke Kahanamoku Lagoon,The Royal Hawaiian,Iolani Palace,Aliʻiōlani Hale,Diamond Head Crater Hike,Hanauma Bay,Tantalus Lookout,Lyon Arboretum,Moanalua Gardens,Pearl Harbor,Ko Olina Lagoons,Halona Blowhole Lookout,Makapu'u Lookout,Bellows Beach,Lanikai Beach,Ho'omaluhia Botanical Garden,Byodo-In Temple,Macadamia Nut Farm Outlet,Kualoa Ranch,Kualoa Beach Park,Dole Pineapple Plantation,Historic Hale'iwa Town,Turtle Beach,Waimea Bay Beach,Banzai Pipeline,Sunset Beach,Waimea Falls,Manoa Falls,Lulumahu Falls,Jackass Ginger Pool,Likeke Falls,Kahana Bay Beach Park,Malaekahana Beach,Sandy Beach,Pokai Bay,Ala Moana Beach Park,Makapuu Beach Park,Bellows Field Beach Park,Makaha Beach Park,Diamond Head Beach Park,Nanakuli Beach Park
0,51017586,Kihei/Wailea,20.68985,-156.43611,24999.0,181.275790,185.596359,181.278679,188.650675,110.999200,139.112554,134.960714,117.384313,165.935984,165.583882,165.748153,177.224418,175.476712,164.398033,111.650145,173.510801,98.231133,98.404286,117.465172,117.383356,179.262978,135.592491,134.967566,135.742585,107.937098,100.734956,113.399653,188.646325,118.870254,124.386841,102.006314,98.686783,198.196195,97.281829,101.416582,70.916060,55.494632,51.525959,59.519523,58.867104,51.015276,55.959290,65.208398,43.396738,58.813765,103.228458,103.228458,64.175962,...,49.155314,8.908354,37.122808,1.049726,40.654774,38.979170,34.047555,14.294331,49.931830,34.103774,38.321926,165.902881,167.327849,165.388121,170.251523,170.336478,162.956378,151.808705,166.797591,165.399957,175.409211,180.591605,198.444522,150.637015,149.731722,157.089114,159.342745,168.008122,172.692049,178.315473,177.431364,177.328811,197.522056,206.692208,202.284236,205.403579,205.588376,204.826497,204.268418,165.534775,167.280899,167.940009,165.983179,182.860303,194.931543,150.335980,209.323430,168.470707,150.137917,157.089114,214.052747,162.964757,202.320451
1,147713325,Leeward Side,21.34109,-158.11871,24999.0,370.637526,375.633334,370.657484,367.568127,308.020218,332.969293,309.554943,293.076336,362.878895,362.499143,362.710534,373.071825,371.127631,361.825018,308.338023,370.362297,289.857268,288.321787,293.092954,293.073763,375.324216,309.340757,308.496443,309.664661,302.570185,296.117954,288.851723,367.564363,294.570900,299.476704,281.273111,289.980801,368.157886,289.236519,282.202797,127.621959,143.358528,148.979276,140.242516,142.635584,149.631332,142.766876,133.420983,155.272017,154.620381,97.889499,97.889499,140.987115,...,241.893100,194.525183,163.890339,198.060698,165.152736,164.401046,222.262078,201.995170,245.211984,165.921779,173.055809,33.007115,31.621752,33.489889,29.097777,28.971037,35.865947,47.798041,33.006515,35.259845,25.024092,20.368697,1.368647,49.455